<a href="https://colab.research.google.com/github/haruhiysd/QM-Group-Project/blob/main/quarterly_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install Basemap
!pip install ipyleaflet
!pip install geopandas folium ipywidgets openpyxl
!pip install folium ipywidgets pandas
import pandas as pd
import folium
import json
import geopandas as gpd
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from ipywidgets import interactive
from IPython.display import display
import matplotlib.pylab as pylab
from datetime import date

pylab.rcParams['figure.figsize'] = (10, 8)

In [5]:
df=pd.read_csv('/merged df.csv')

In [6]:
rename_dict = {
    '01 Andalucía': 'Andalucía',
    '02 Aragón': 'Aragón',
    '03 Asturias, Principado de': 'Principado de Asturias',
    '04 Balears, Illes': 'Illes Balears',
    '05 Canarias': 'Canarias',
    '06 Cantabria': 'Cantabria',
    '07 Castilla y León': 'Castilla y León',
    '08 Castilla - La Mancha': 'Castilla-La Mancha',
    '09 Cataluña': 'Cataluña',
    '10 Comunitat Valenciana': 'Comunitat Valenciana',
    '11 Extremadura': 'Extremadura',
    '12 Galicia': 'Galicia',
    '13 Madrid, Comunidad de': 'Comunidad de Madrid',
    '14 Murcia, Región de': 'Región de Murcia',
    '15 Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra',
    '16 País Vasco': 'País Vasco',
    '17 Rioja, La': 'La Rioja',
}

In [7]:
df['Period'] = pd.to_datetime(df['Period'], dayfirst=True, errors='coerce')

df_filtered = df[
    (df['Period'] >= '2015-01-01') &
    (df['Period'] <= '2023-10-01')
].copy()

In [8]:
df_filtered = df_filtered[[
        'Autonomous_community',
        'Period',
        'Monthly_number_of_tourists',
        'Unemployment_rate',
        'HPI',
        'total_waste_capita',
        'Total_mixed_waste',
        'Total_waste_no_CDW_and_other',
]].copy()

In [9]:
df_filtered = df_filtered[df_filtered['Period'].dt.month.isin([1, 4, 7, 10])]

In [12]:
df_filtered['Autonomous_community'] = (
    df_filtered['Autonomous_community']
    .replace(rename_dict)
)

In [29]:
gdf_spain = gpd.read_file('/georef-spain-comunidad-autonoma.geojson')
gdf_spain

,geo_point_2d,year,acom_code,acom_name,acom_area_code,acom_type,acom_name_local,acom_iso3166_code,geometry
0,"{ ""lon"": -2.9506895373799118, ""lat"": 35.291074...",2022,19,Ciudad Autónoma de Melilla,ESP,autonomous communities,None,ML,"POLYGON ((-2.95264 35.3203, -2.96005 35.31595,..."
1,"{ ""lon"": -3.7170278269539754, ""lat"": 40.495064...",2022,13,Comunidad de Madrid,ESP,autonomous communities,None,MD,"MULTIPOLYGON (((-3.53972 41.16504, -3.5439 41...."
2,"{ ""lon"": -1.6460208786922079, ""lat"": 42.667166...",2022,15,Comunidad Foral de Navarra,ESP,autonomous communities,None,NC,"MULTIPOLYGON (((-2.42058 42.48923, -2.41567 42..."
3,"{ ""lon"": -4.7818570800084954, ""lat"": 41.754353...",2022,07,Castilla y León,ESP,autonomous communities,None,CL,"MULTIPOLYGON (((-6.98576 41.97104, -6.98354 41..."
4,"{ ""lon"": -5.9933663607595538, ""lat"": 43.292377...",2022,03,Principado de Asturias,ESP,autonomous communities,None,AS,"MULTIPOLYGON (((-7.03319 43.54473, -7.03371 43..."
5,"{ ""lon"": -3.2157744842782665, ""lat"": 35.353164...",2022,20,Territorio no asociado a ninguna autonomía,ESP,autonomous communities,None,None,"MULTIPOLYGON (((-4.29789 35.17076, -4.30014 35..."
6,"{ ""lon"": -3.0045800643630733, ""lat"": 39.580935...",2022,08,Castilla-La Mancha,ESP,autonomous communities,None,CM,"MULTIPOLYGON (((-5.04695 38.72909, -5.04179 38..."
7,"{ ""lon"": -4.0293308511606991, ""lat"": 43.197702...",2022,06,Cantabria,ESP,autonomous communities,None,CB,"MULTIPOLYGON (((-4.84102 43.18078, -4.84249 43..."
8,"{ ""lon"": 1.5288177458229948, ""lat"": 41.7984138...",2022,09,Cataluña,ESP,autonomous communities,Catalunya,CT,"MULTIPOLYGON (((0.5147 40.52305, 0.51495 40.52..."
9,"{ ""lon"": -5.3430527324121435, ""lat"": 35.893586...",2022,18,Ciudad Autónoma de Ceuta,ESP,autonomous communities,None,CE,"POLYGON ((-5.38206 35.9124, -5.37983 35.91049,..."


In [41]:
import geopandas as gpd
filtered_gdf = gdf_spain[~gdf_spain['acom_name'].apply(lambda x: x in ["Ciudad Autónoma de Ceuta", "Ciudad Autónoma de Melilla	", "Territorio no asociado a ninguna autonomía"])]
filtered_gdf.to_file("filtered_spain.geojson", driver="GeoJSON")

In [42]:
filtered_gdf.rename(
    columns={
        'acom_name': 'Autonomous_community',
        'year': 'Period',
    },
    inplace=True
)

<ipython-input-42-cb74d62583c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_gdf.rename(


In [43]:
filtered_gdf

,geo_point_2d,Period,acom_code,Autonomous_community,acom_area_code,acom_type,acom_name_local,acom_iso3166_code,geometry
0,"{ ""lon"": -2.9506895373799118, ""lat"": 35.291074...",2022,19,Ciudad Autónoma de Melilla,ESP,autonomous communities,None,ML,"POLYGON ((-2.95264 35.3203, -2.96005 35.31595,..."
1,"{ ""lon"": -3.7170278269539754, ""lat"": 40.495064...",2022,13,Comunidad de Madrid,ESP,autonomous communities,None,MD,"MULTIPOLYGON (((-3.53972 41.16504, -3.5439 41...."
2,"{ ""lon"": -1.6460208786922079, ""lat"": 42.667166...",2022,15,Comunidad Foral de Navarra,ESP,autonomous communities,None,NC,"MULTIPOLYGON (((-2.42058 42.48923, -2.41567 42..."
3,"{ ""lon"": -4.7818570800084954, ""lat"": 41.754353...",2022,07,Castilla y León,ESP,autonomous communities,None,CL,"MULTIPOLYGON (((-6.98576 41.97104, -6.98354 41..."
4,"{ ""lon"": -5.9933663607595538, ""lat"": 43.292377...",2022,03,Principado de Asturias,ESP,autonomous communities,None,AS,"MULTIPOLYGON (((-7.03319 43.54473, -7.03371 43..."
6,"{ ""lon"": -3.0045800643630733, ""lat"": 39.580935...",2022,08,Castilla-La Mancha,ESP,autonomous communities,None,CM,"MULTIPOLYGON (((-5.04695 38.72909, -5.04179 38..."
7,"{ ""lon"": -4.0293308511606991, ""lat"": 43.197702...",2022,06,Cantabria,ESP,autonomous communities,None,CB,"MULTIPOLYGON (((-4.84102 43.18078, -4.84249 43..."
8,"{ ""lon"": 1.5288177458229948, ""lat"": 41.7984138...",2022,09,Cataluña,ESP,autonomous communities,Catalunya,CT,"MULTIPOLYGON (((0.5147 40.52305, 0.51495 40.52..."
10,"{ ""lon"": -15.667166418179175, ""lat"": 28.341572...",2022,05,Canarias,ESP,autonomous communities,None,CN,"MULTIPOLYGON (((-17.96104 27.69253, -17.9584 2..."
11,"{ ""lon"": -6.1506507747552881, ""lat"": 39.191430...",2022,11,Extremadura,ESP,autonomous communities,None,EX,"POLYGON ((-6.93156 38.20821, -6.93048 38.20699..."


In [15]:
gdf_spain = gdf_spain[['Autonomous_community', 'geometry']].copy()

In [26]:
def create_choropleth_map(year_month, metric_column, legend_label, max_value=None, min_value=None):

    year = year_month.year
    month = year_month.month


    df_year = df_filtered[
        (df_filtered['Period'].dt.year == year) &
        (df_filtered['Period'].dt.month == month)
    ].copy()


    print("Columns in gdf_spain:", gdf_spain.columns)
    print("Columns in df_year:", df_year.columns)


    gdf_spain['Period'] = pd.to_datetime(gdf_spain['Period'], format='%Y')
    gdf_spain['Period'] = year_month.year

    merged_gdf = gdf_spain.merge(
        df_year,
        on=['Autonomous_community', 'Period'],
        how='left'
    )
    merged_gdf['Period'] = merged_gdf['Period'].dt.strftime('%Y-%m-%d')
    merged_gdf[metric_column] = pd.to_numeric(merged_gdf[metric_column], errors='coerce')
    merged_gdf[metric_column] = merged_gdf[metric_column].fillna(0)

    m = folium.Map(location=[40, -4], zoom_start=6)

    if min_value is not None and max_value is not None:
        step = (max_value - min_value) / 5
        bins = list(range(int(min_value), int(max_value) + 1, int(step)))
        merged_gdf[metric_column] = merged_gdf[metric_column].clip(lower=min_value, upper=max_value)
    elif max_value is not None:
        step = max_value / 5
        bins = list(range(0, int(max_value) + 1, int(step)))
        merged_gdf[metric_column] = merged_gdf[metric_column].clip(lower=0, upper=max_value)


    folium.Choropleth(
        geo_data=merged_gdf.__geo_interface__,
        data=merged_gdf,
        columns=['Autonomous_community', metric_column],
        key_on='feature.properties.Autonomous_community',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.5,
        highlight=True,
        threshold_scale=bins,
        legend_name=legend_label
    ).add_to(m)

    folium.GeoJson(
        merged_gdf,
        name='Regions',
        style_function=lambda x: {
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['Autonomous_community', metric_column],
            aliases=['Region', metric_column]
        )
    ).add_to(m)
    m.save("my_map.html")

    return m



In [27]:
@widgets.interact(
    Period=widgets.SelectionSlider(
        options=[date(year, month, 1) for year in range(2015, 2024) for month in [1, 4, 7, 10]],
        description='Period:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
)
def show_international_all(Period=date(2015, 1, 1)):
    return create_choropleth_map(
        Period,
        metric_column='HPI',
        legend_label='HPI',
        max_value=200
    )

interactive(children=(SelectionSlider(continuous_update=False, description='Period:', options=(datetime.date(2…